# Library
[Naive vs Advance  Rags Blog](https://medium.com/@myscale/naive-rag-vs-advanced-rag-17b38cda44c1)

In [1]:
import os
os.chdir("../")

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.llms.llamafile import Llamafile
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import torch


# Load Emberding Model

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',
                                      model_kwargs = {'device': device})

C:\Users\User\AppData\Local\Temp\ipykernel_10580\3511400781.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',


# Load Vector DB

In [4]:
vectorstore = FAISS.load_local(folder_path="vector_db/faiss_index_hybride_all_books",
                               embeddings=embedding_model,
                               allow_dangerous_deserialization=True)

In [5]:
# Step 2: Create a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})    

# Load LLM


In [6]:
llm = Llamafile()

In [7]:
llm.invoke("Tell me a joke")

". Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nHow was that? Do you want to hear another one?<|eot_id|>"

In [8]:
query = "Tell me a joke"

for chunks in llm.stream(query):
    print(chunks, end="")

print()

 about a tree
Here's one:
Why did the tree go to the party?

(Wait for it...)

Because it wanted to branch out and meet new people!

Hope that made you leaf through your laughter! (Sorry, I couldn't resist!)<|eot_id|>


# Prompt

In [9]:
# Instantiation using from_template (recommended)
# prompt = PromptTemplate.from_template("Say {foo}")
# prompt.format(foo="bar")

In [10]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Tolkien scholar assistant."),
    ("system", "Use the provided LOTR context to answer the question with citations."),
    ("human", "Context: {context}\n\nQuestion: {query}\nAnswer:")
])

# Complitions

In [ ]:
# # Create QA chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     return_source_documents=True,
#     chain_type_kwargs={
#         'prompt': prompt
#     }
# )

In [11]:

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
qa_chain = create_retrieval_chain(retriever, combine_docs_chain)


In [12]:
query = 'what is Narsil?'


In [ ]:
response = qa_chain.invoke({"input": query, "query": query})

print("RESULT: ", response["answer"])
print("SOURCE DOCUMENTS: ", response["context"])

In [13]:
for chunk in qa_chain.stream({"input": query, "query": query}):
    if "answer" in chunk:
        print(chunk["answer"], end="", flush=True)

print()

 Narsil is a sword, specifically the sword of Elendil that was broken and had its light extinguished, but it has not yet been forged again. (From various citations: 231, 41, 160, 41) 
Narsil's name is composed of two basic stems without variation or adjuncts: √NAR 'fire' & √THIL 'white light'. This symbolises the shards of the sword that was broken, and its name is thus associated with fire and light. (From context: Ernil i Pheriannath III 41 cf. 231) 

Note: As a Tolkien scholar assistant, you may need to use your knowledge of the texts to provide additional information or insights about Narsil.
Also note that the names mentioned in this conversation are from J.R.R. Tolkien's Middle-earth legendarium.<|eot_id|>
